# Chart generations Notebook

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot

In [2]:
offline.init_notebook_mode(connected=True)

## Bubble Chart for Emissions Intensity


In [3]:
def get_company_id(corp_name):
    XLSX_PATH = os.path.join ('../django_project/static/django_project', 'data', 'sp100_data.xlsx')
    df= pd.read_excel(
        XLSX_PATH, 
        sheet_name = "companies",
        engine = 'openpyxl', 
        usecols = ['company_id', 'company_name'],
        )
    return df.loc[df['company_name']==corp_name].iloc[0,0]

In [4]:
corp_name = "3M"
company_id = get_company_id(corp_name)

# Step 1: Set up the data sources

In [5]:
XLSX_PATH = os.path.join ('../django_project/static/django_project', 'data', 'sp100_data.xlsx')
COLS_TO_USE = {
    'GHG17':['company_id', 'gross_total_scope1', 'gross_loc_scope2', 'gross_mkt_scope2', 'gross_total_scope3'],
    'GHG18':['company_id', 'gross_total_scope1', 'gross_loc_scope2', 'gross_mkt_scope2', 'gross_total_scope3'],
    'GHG19':['company_id', 'gross_total_scope1', 'gross_loc_scope2', 'gross_mkt_scope2', 'gross_total_scope3'],
}


# Step 2: Merge the tables

In [6]:

df_by_year = pd.read_excel(
    XLSX_PATH, 
    sheet_name = "GHG17",
    engine = 'openpyxl', 
    usecols = ['company_id', 'gross_total_scope1', 'gross_loc_scope2', 'gross_mkt_scope2', 'gross_total_scope3'],
    )

In [7]:
years = ["2017", "2018", "2019"]


In [8]:
i=0
all_df = []
for sheetname, cols in COLS_TO_USE.items():
    
    df_by_year = pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        )
    df_by_year = df_by_year.loc[df_by_year['company_id'] == company_id]
    df_by_year['year'] = years[i]

    all_df.append(df_by_year)
    i+=1

In [9]:
    merged_df = pd.concat(
        all_df
    )

In [10]:
merged_df.head()

,company_id,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,gross_total_scope3,year
0,1.0,3960000,1650000,1650000,9280000,2017
0,1.0,4790000,1480000,1480000,9530000,2018
0,1.0,4050000,1780000,1320000,8.9258e+06,2019


In [16]:
bar_data = merged_df[['gross_total_scope1','gross_loc_scope2', 'gross_total_scope3']]

In [17]:
bar_data

,gross_total_scope1,gross_loc_scope2,gross_total_scope3
0,3960000,1650000,9280000
0,4790000,1480000,9530000
0,4050000,1780000,8.9258e+06


# Extract each data serie

In [11]:
scope1 = merged_df['gross_total_scope1']
scope2 = merged_df['gross_loc_scope2']
scope3 = merged_df['gross_total_scope3']


In [12]:
layout = go.Layout (
    title = '<b>GHG Emissions Evolution</b><br>',
    title_x = 0.5,
    titlefont = dict(
        family = 'Arial',
        size = 16),
    plot_bgcolor = 'antiquewhite',
    xaxis =  dict(
        title = '<b>Reporting years</b>'),
    yaxis = dict(
        title = 'Revenue'),
)

In [18]:
"""
trace1 = go.Scatter(x=years, 
                    y=scope1, 
                    showlegend = True, 
                    name='Scope 1',
                    mode = "lines",
                    line = dict(color='#de425b', 
                                width=2))

trace2 = go.Scatter(x=years, 
                    y=scope2, 
                    showlegend = True, 
                    name='Scope 2',
                    mode = "lines",
                    line = dict(color='#feba65', 
                                width=2))

trace3 = go.Scatter(x=years, 
                    y=scope3, 
                    showlegend = True, 
                    name='Scope 3',
                    mode = "lines",
                    line = dict(color='#488f31', 
                                width=2))

"""
import plotly.express as px

fig = px.bar(bar_data)
                
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=80,
        pad=4
    ),
    paper_bgcolor="#bad0af",
    plot_bgcolor = '#f1f1f1',
)
config = {'displaylogo': False}


In [19]:
offline.iplot(fig, config = config)

In [15]:
name_fig = "ghg_linechart_{}".format(corp_name)
#fig.write_image("../django_project/static/django_project/images/charts/{}.svg".format(name_fig), scale=3, height = 300)
plot(fig, config=config, filename = '../django_project/templates/django_project/corporates/charts/html_exports/{}.html'.format(name_fig), auto_open=False)

'../django_project/templates/django_project/corporates/charts/html_exports/ghg_linechart_3M.html'